## Parâmetros

In [1]:
p = 761
q = 4591
t = 143

Zx.<x> = ZZ[]
R.<xp> = Zx.quotient(x^p - x - 1)
print(R)

F3 = GF(3)
F3x.<x3> = F3[]
R3.<xp3> = F3x.quotient(x^p - x - 1)
print(R3)

Fq = GF(q)
Fqx.<xq> = Fq[]
Rq.<xqp> = Fqx.quotient(x^p - x - 1)
print(Rq)

Univariate Quotient Polynomial Ring in xp over Integer Ring with modulus x^761 - x - 1
Univariate Quotient Polynomial Ring in xp3 over Finite Field of size 3 with modulus x3^761 + 2*x3 + 2
Univariate Quotient Polynomial Ring in xqp over Finite Field of size 4591 with modulus xq^761 + 4590*xq + 4590


### Validação dos Parâmetros

In [2]:
def params_validation():
    try:
        assert p.is_prime()
        #print('p primo!')
        assert q.is_prime()
        #print('q primo!')
        assert t > 1
        #print('t>1!')
        assert p > 3*t
        #print('p>3t')
        assert q > 32*t + 1
        #print('q>32t + 1')
        assert(xq^p-xq-1).is_irreducible()
        #print('polinomio irredutivel')
    except:
        print('Parâmetros inválidos!')
        return
    print('Parâmetros válidos!')

params_validation()

Parâmetros válidos!


## Métodos de transformação

In [63]:
q12 = 2295 # q12 = (q//2)

def ZZ_fromFq(u):
    assert u in Fq
    return ZZ(u + 2295) - 2295

def ZZ_fromF3(u):
    assert u in F3
    return ZZ(u + 1) - 1

def Rq_fromR(r):
    assert r in R
    ret = Rq([r[i] for i in range(p)])
    assert ret in Rq
    return ret

def R3_fromR(r):
    assert r in R
    return R3([r[i] for i in range(p)])

def R_fromRq(r):
    assert r in Rq
    return R([ZZ_fromFq(r[i]) for i in range(p)])

def R_fromR3(r):
    assert r in R3
    return R([ZZ_fromF3(r[i]) for i in range(p)])

def RoundRq(a):
    assert a in Rq
    c = R_fromRq(a)
    r = [3*round(c[i]/3) for i in range(p)]
    assert all([abs(r[i]-c[i])<=1 for i in range(p)])
    r = R(r)
    assert Rounded_is(r)
    return r

def RoundR3(a): # r in {0,1,-1} with u-r in {...,-3,0,3,...}
    assert a in R3
    r = [nicemod3(lift(gri)) for gri in list(a)]
    r = R(r)
    return r

def nicelift(u):
    return lift(u + q12) - q12
    
def nicemod3(u): # r in {0,1,-1} with u-r in {...,-3,0,3,...}
    return u - 3*round(u/3)



## Funções de verificação

In [64]:
'''
Função que verifica se um elemento de R é Small:
    Todos os coeficientes em {-1,0,1}
'''
def is_Small(r):
    assert r in R
    return all( abs(r[i]) <= 1 for i in range(p) )

'''
smallElems = [-1,0,1]
def is_Small(elementR):
    l = list(elementR)
    for e in l:
        if(not(e in smallElems)):
            return False
    return True
'''

'''
Função que verifica se um elemento tem Hamming Weight de 2t :
    (#coeficientes != 0) == (2*t)
'''
def is_HammingWeight(r):
    assert r in R
    return (2*t) == len([i for i in range(p) if r[i] != 0])

'''
Função que verifica se um elemento de R é tsmall:
    Small and HammingWeight
'''
def is_tSmall(r):
    assert r in R
    return is_Small(r) and is_HammingWeight(r)

'''
Função que verifica se um elemento de R foi rounded com sucesso
'''
def Rounded_is(r):
    assert r in R
    return( all(r[i]%3 == 0 for i in range(p)) and
            all(r[i]>=-q12 for i in range(p)) and
            all(r[i]<=q12 for i in range(p)))



## Métodos de codificação

In [65]:
import hashlib
import itertools

def concat(l):
    return list(itertools.chain.from_iterable(l))

def hash(s):
    h = hashlib.sha512()
    h.update(s)
    return h.digest()


q12 = 2295 # (q12 = ((q-1)/2))

In [66]:
# --------------------------------------------------- STR->INT ---------------------------------------------------- #
'''
Transforma uma string num inteiro.
'''
def str2int(s):
    return sum(ord(s[i])*256^i for i in range(len(s)))

'''
u: Lista de inteiros convertidos das strings
     for i in range(0,len(seq),nbytes) -> divide a sequencia em intervalos de nbytes para ser iterado
     str2int(s[i:i+nbytes]) -> converte a string desde a posição i até à posição i+nbytes em inteiro
return: 
    for i in range(len(u)) -> iterar sobre cada inteiro convertido.
        for j in range(batch) -> itera desde 0 até batch
             mod (u[i]//radix^j, radix)
'''
def str2seq(s,radix,batch,bytes):
    u = [str2int(s[i:i+bytes]) for i in range(0,len(s),bytes)]
    return concat([(u[i]//radix^j)%radix for j in range(batch)] for i in range(len(u)))

# --------------------------------------------------- STR->INT ---------------------------------------------------- #

In [67]:
# --------------------------------------------------- INT->STR ---------------------------------------------------- #
'''
Este método transforma um inteiro em uma string de *nbytes* bytes.

'''
def int2str(u,nbytes):
    return ''.join(chr((u//256^i)%256) for i in range(nbytes))

'''
Este método é responsável por codificar blocos de sequencia. 

Ele vai percorrer a lista de coeficientes que recebe iterando sempre de batch em batch elementos.
(com o: for i in range(0,len(u),batch)) 

Por cada iteração acima referida, vamos iterar sobre cada elemento no intervalo de 2 iterações. Em cada um deles
vamos calcular a soma -> u[i] + u[i+1]*radix + u[i+2]*radix^2 + ... + u[i+t]*radix^t

Depois dessa soma ser calculada é convertida em n bytes.

No final vamos obter toda a sequencia codificada em string.

'''     
def seq2str(u,radix,batch,n): # radix^batch <= 256^bytes    
    return ''.join(int2str(sum(u[i+t]*radix^t for t in range(batch)),n)
                   for i in range(0,len(u),batch))

# --------------------------------------------------- INT->STR ---------------------------------------------------- #

In [68]:
# ------------------------------------------------------ Rq ------------------------------------------------------- #
'''
Este método começa por transformar os elementos de h (que estão em Rq).
Vamos transformar esses elementos de forma a eles estarem no intervalo [0,4590]:
    Recebe o h e transforma os elementos no intervalo ]-(q/2)-1, (q/2)-1[.
    Adiciona q/2.

Após essa tranformação, pegamos na lista resultante(h1) e convertemos em string.
Como os últimos 6 bytes são 0, suprimimos esse valor devolvendo os 1218 bytes.
'''
def encodeRq(h):
    h1 = [q12 + nicelift(h[i]) for i in range(p)]+[0]*(-p % 5)
    return seq2str(h1,6144,5,8)[:1218]

'''
Este método começa por transformar a sequencia em uma lista de inteiros.
Depois verifica se a lista tem algum valor fora dos admitiveis
Por fim percorre essa lista e subtrai por q/2 (para anular a soma efectuada no encode)
'''
def decodeRq(hstr):
    h = str2seq(hstr,6144,5,8)
    if max(h) >= q: 
        raise Exception("pk out of range")
    return Rq([h[i]-q12 for i in range(p)])

# ------------------------------------------------------ Rq ------------------------------------------------------- #

In [69]:
# ------------------------------------------------------ Zx-------------------------------------------------------- #
'''
Este método é responsável por codificar um elemento de Zx. 
Para tal vamos pegar no valor que recebemos e a cada coeficiente vamos adicionar 1 de forma a obtermos coeficientes no
intervalo [0,1,2]. 
Após isso vamos escrever um conjunto de 4 elementos em radix 4, obtendo assim um byte.
'''
def encodeZx(m): # assumes coefficients in range {-1,0,1}
    m = [m[i]+1 for i in range(p)] + [0]*(-p % 4)  
    return seq2str(m,4,4,1)


def decodeZx(mstr):
    m = str2seq(mstr,4,4,1)  
    return Zx([m[i]-1 for i in range(p)])
# ------------------------------------------------------ Zx-------------------------------------------------------- #

In [70]:
# --------------------------------------------------- RoundRq------------------------------------------------------ #
q61 = ZZ((q-1)/6)
'''
Neste método vamos codificar elementos de rounded rings.
'''
def encoderoundedRq(c):
    c = [q61 + nicelift(c[i]/3) for i in range(p)] + [0]*(-p % 6)
    #print(c)
    return seq2str(c,1536,3,4)[:1015]

def decoderoundedRq(cstr):
    c = str2seq(cstr,1536,3,4)
    if max(c) > 1530: 
        raise Exception("c out of range")
    c = [ci%(q61*2+1) for ci in c]
    return 3*Rq([c[i]-q61 for i in range(p)])
# --------------------------------------------------- RoundRq------------------------------------------------------ #

## Geradores

In [71]:
def random8(): 
    return randrange(256)

'''
c0 + 256c1 + 256^2 * c2 + 256^3 *c3
'''
def urandom32():
    c0 = random8()
    c1 = random8()
    c2 = random8()
    c3 = random8()
    return c0 + 256*c1 + 65536*c2 + 16777216*c3

def random32even(): return urandom32() & (-2)
def random321mod4(): return (urandom32() & (-3)) | 1

def randomrange3():
    return ((urandom32() & 0x3fffffff) * 3) >> 30

def random_small():
    r = R([randomrange3()-1 for i in range(p)])
    assert is_Small(r)
    return r

def randomg():
    g = Zx([randomrange3()-1 for i in range(p)])
    assert R3(g).is_unit()
    return g
def random_tSmall():
    L = [random32even() for i in range(2*t)]
    L += [random321mod4() for i in range(p-2*t)]
    L.sort()
    L = [(L[i]%4)-1 for i in range(p)]
    return Zx(L)


def List_to_tSmall(L):
    w = 2*t
    L = [L[i]&-2 for i in range(w)] + [(L[i]&-3)|1 for i in range(w,p)]
    assert all(L[i]%2 == 0 for i in range(w))
    assert all(L[i]%4 == 1 for i in range(w,p))
    L.sort()
    L = [(L[i]%4)-1 for i in range(p)]
    assert all(abs(L[i]) <= 1 for i in range(p))
    assert sum(abs(L[i]) for i in range(p)) == w
    r = R(L)
    assert is_tSmall(r)
    return r

def generateG():
    while True:
        g = randomg()
        print('GenerateG: Random Small Gerado')
        if R3_fromR(g).is_unit(): 
            print('GenerateG: É irredutível em R3')
            break
        else:
            print('GenerateG: Não é irredutível...')
    return g

## KeyGen

In [75]:
def keyGen():
    g = generateG()
    print('_KeyGen_: Temos **g**.')
    
    #inv_g_R3 = 1/R3(g) 
    inv_g = 1/R3(g)
    #s2 = [nicemod3(lift(gri)) for gri in list(inv_g_R3)]
    
    f = random_tSmall()
    print('_KeyGen_: Temos **f**.')
    
    h = Rq(g)/(3*Rq(f))
    print('_KeyGen_: Temos **h**.')
    #print(max(h))
    
    pk = encodeRq(h)
    #print(pk)
    print('\n_KeyGen_: Temos **pk**.')
    
    encoded_f = encodeZx(f) 
    encoded_inv_g = encodeZx(RoundR3(inv_g))
    
    
    secret = encoded_f + encoded_inv_g + pk

    print('_KeyGen_: Temos o **segredo**.')
    return pk, secret
    
        
#test()
pk, secret = keyGen()

GenerateG: Random Small Gerado
GenerateG: É irredutível em R3
_KeyGen_: Temos **g**.
_KeyGen_: Temos **f**.
_KeyGen_: Temos **h**.

_KeyGen_: Temos **pk**.
_KeyGen_: Temos o **segredo**.


In [76]:
def encapsulate(pk):
    h = decodeRq(pk)
    print('_Encapsulate_: Temos **h**.')
    
    r = random_tSmall()
    print('_Encapsulate_: Temos **r**.')
    #print(r)
    
    hr = h*Rq_fromR(r)
    print('_Encapsulate_: Temos **hr**.')
    #print(list(hr))

    m = Zx([-nicemod3(nicelift(hr[i])) for i in range(p)]) #m é hr transformado em {-1,0,1}
    
    c = Rq(m) + hr
    #print(c)
    print('_Encapsulate_: Temos **c**.')
    
    hashR = hash(encodeZx(r))
    
    C = hashR[0:32]
    print('_Encapsulate_: Temos **Confirmation C**.')
    
    K = hashR[32:]
    print('\n_Encapsulate_: Temos **Secret Key K**.')
    
    encoded_c = encoderoundedRq(c)
    print('_Encapsulate_: Temos **encoded c**.')
    
    return C + encoded_c, K

cipherText, sessionKey = encapsulate(pk)

#cipherText = C + encoded_c
#sessionKey = K

_Encapsulate_: Temos **h**.
_Encapsulate_: Temos **r**.
_Encapsulate_: Temos **hr**.
_Encapsulate_: Temos **c**.
_Encapsulate_: Temos **Confirmation C**.

_Encapsulate_: Temos **Secret Key K**.
_Encapsulate_: Temos **encoded c**.


In [78]:
#cipherText <- C + encoded_c
# secret <- encoded_f + encoded_g_inv + pk
def decapsulate(cipherText,secret):
    f = decodeZx(secret[:191]) #pegamos em f
    print('_Decapsulate_: Temos **f**.')
    ginv = decodeZx(secret[191:382]) #pegamos no inverso de g
    pk = secret[382:] #descodificamos e obtemos h
    h = decodeRq(pk)
    print('_Decapsulate_: Temos **h**.')
    
    C = cipherText[0:32]
    print('_Decapsulate_: Temos **Confirmation C**.')
    
    rounded_c = cipherText[32:]
    print('_Decapsulate_: Temos **Rounded c**.')
    
    c = decoderoundedRq(rounded_c)
    print('_Decapsulate_: Temos **c**.')
    
    cX3f = c * Rq_fromR(3*f)
    cX3f = [nicelift(cX3f[i]) for i in range(p)]
    
    r = R3_fromR(ginv) * R3(cX3f)
    r = Zx([nicemod3(lift(r[i])) for i in range(p)])
    print('_Decapsulate_: Temos **r**.')
    
    hr = h * Rq(r)
    m = Zx([-nicemod3(nicelift(hr[i])) for i in range(p)])
    checkc = Rq(m) + hr
    #print(checkc)
    
    fullkey = hash(encodeZx(r))
    if sum(r[i]==0 for i in range(p)) != p-2*t:
        print('erro no sum')
        return False
    if checkc != c: 
        return False
    if fullkey[:32] != C: 
        return False
    return fullkey[32:]
    
    
decapsulate(cipherText,secret)

_Decapsulate_: Temos **f**.
_Decapsulate_: Temos **h**.
_Decapsulate_: Temos **Confirmation C**.
_Decapsulate_: Temos **Rounded c**.
_Decapsulate_: Temos **c**.
_Decapsulate_: Temos **r**.


'>\x9df. \xd9\xaa\x99\xf3\x98" ]\xd9|\xe6\x0cR1\xa7\xc3\xb2\xfa\xc3\xce\xaf\xf4\xac\xe5\xd0\xe3\xf8'